# Tutorial 3: Estimating a Simple Mean

In this notebook we will set up the *simplest possible inference problem* and solve it with the same algorithm, but while storing the data in two different places: RAM, and a database.

Suppose we have a catalog of stars, each one with an ID and a measured distance. We will assume that they belong to a single population lying at the same distance, and that they are standard candles (so that we have noisy estimates of the distance to each star). Our model has only one parameter, the mean distance to the population.

Goals:

* Simulate a single stellar population, and generate an observed distance dataset
* Load the data in to memory and estimate the mean distance 
* Create a simple database and store the data in it
* Load the data from the database and estimate the mean distance in a few different ways, and compare the time taken and the memorry used. 

### Requirements

You will need to install `sqlalchemy`, and we'll be using `pandas` as well.


In [1]:
import numpy as np
import pandas as pd
import os

import sqlalchemy as sq
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import func

## Probability Theory

We have measured distances $d_k$ for each star $k$. We are looking for the posterior PDF for the mean distance $\mu$:

${\rm Pr}(\mu | \boldsymbol{d}) = {\rm Pr}(\mu) \prod_k {\rm Pr}(d_k | \mu)$

We will assign a uniform prior for $\mu$ and etc etc etc. We're just going to take the arithmetic mean of $d$ and call it $\mu$.

## Simulating a Stellar Population

In [2]:
class StellarPopulation(object):
    """
    Simulate a population of stars, all with the same distance.
    """
    def __init__(self, distance):
        """
        Instantiate a StellarPopulation object.
        
        Parameters
        ----------
        distance : float
            Scalar distance to the cluster of stars, in kpc
        """
        self.distance = distance 
        return
    
    def generate(self, N=1000,rms_error=0.1):
        """
        Simulates the observations of N stars.
        
        Parameters
        ----------
        N : int
            Number of stars to generate
        rms_error : float
            RMS fractional distance uncertainty
        """
        self.N = N
        self.rms_error = rms_error
        self.d_obs = self.distance + (self.rms_error * self.distance) * np.random.randn(self.N)
        self.id = map(str, range(self.N))
        return
    
    def estimate_mean_distance(self):
        """
        Estimate the mean of the observed stellar distances, and return this as well as the wallclock time taken.
        
        Returns
        -------
        mu : float
            Estimated mean distance in kpc
        time : float
            Wallclock time in milliseconds
        """
        import time as wallclock
        start = wallclock.time()
        mu = np.mean(self.d_obs)
        end = wallclock.time()
        time = (end-start) * 1e3
        return mu, time

In [3]:
N_stars_per_cluster = 100000

In [4]:
cluster = StellarPopulation(3.0)
cluster.generate(N_stars_per_cluster)

In [5]:
mu, time = cluster.estimate_mean_distance()
print "Estimated mean distance = ", mu
print "Wallclock time spent = ", np.round(time,1), "milliseconds"

Estimated mean distance =  3.00081275195
Wallclock time spent =  0.2 milliseconds


## Setting Up A Database

Let's make two tables, one to store information about stellar populations, and one to store measurements of stars within populations. When these two classes are declared, table metadata is prepared and stored in the `Base` object.

In [6]:
Base = declarative_base()

In [7]:
class Population(Base):
    __tablename__ = 'population'
    
    id = sq.Column(sq.Integer, primary_key=True)
    distance = sq.Column(sq.Float, nullable=False)
    
class Star(Base):
    __tablename__ = 'star'
    
    id = sq.Column(sq.Integer, primary_key=True)
    population_id = sq.Column(sq.Integer, sq.ForeignKey('population.id'), nullable=False)
    distance = sq.Column(sq.Float, nullable=False)

In [8]:
dbfile = 'mean.db'

try: os.remove(dbfile)
except: pass

In [9]:
engine = sq.create_engine('sqlite:///'+dbfile)
Base.metadata.create_all(engine)

Base.metadata.bind = engine
DBSession = sessionmaker(bind=engine)
session = DBSession()

## Inserting Data

Let's make 10 clusters, each with `N_stars_per_cluster` members, but lying at different mean distances `d_c`. The `population` table will then have 10 records.

In [10]:
clusters = []
for k in range(10):
    d_c = float(k)
    clusters.append(StellarPopulation(d_c))
    clusters[k].generate(N_stars_per_cluster)    
    session.add(Population(id=k, distance=d_c))

The `star` table will contain all the distance measurements `d_s` we have, one for each star. If `N_stars_per_cluster = 100000` then this table will contain 1 million rows. Adding them one by one will take a minute or so. The star `id` must be unique, so we increment it by hand. We could just allow the id's to be created automatically, but then they would be 1-indexed.

In [11]:
%%time
j = 0
for k in range(10):
    for d_s in clusters[k].d_obs:
        session.add(Star(id=j, population_id=k, distance=d_s))
        j += 1

CPU times: user 53.9 s, sys: 1.63 s, total: 55.5 s
Wall time: 56.4 s


Now we carry out the transaction, writing all the added records to the database file. This will also take a little while.

In [12]:
%%time
session.commit()

CPU times: user 1min 7s, sys: 2.71 s, total: 1min 10s
Wall time: 1min 12s


In [13]:
!du -h $dbfile

 17M	mean.db


## Querying Data and Estimating the Mean

Let's do this two different ways. First, we'll query for the all the distances to stars in population 3, unpack them and calculate the mean. We'll see that it makes a difference whether we ask for all the stars and then compute their mean distance, if we ask for all stellar distances and then compute their mean. Then, we'll use the inbuilt `SQLite` `avg` function to compute the mean distance from within the database, for comparison. The session, and the classes defining the tables and their entries, are all global objects, accessible to the short function we will use for estimating the mean stellar distance. 

In this function we use the `time` package to measure wallclock time, and `guppy` to measure memory usage.

In [14]:
def estimate_mean(k=3,method='sql_function'):
    
    import time as wallclock
    import guppy
    
    measure = guppy.hpy()
    
    start, end = {}, {}
    start['memory'] = measure.heap().size
    start['time'] = wallclock.time()
    
    stars, distances, mean = None, None, None

    if method == 'query_stars':
        stars = session.query(Star).filter(Star.population_id == k).all()
        mean = np.mean(np.array([s.distance for s in stars]))

    elif method == 'query_distances':
        distances = session.query(Star.distance).filter(Star.population_id == k).all()
        mean = np.mean(np.array([d.distance for d in distances]))
  
    elif method == 'pandas_query_stars':
        df = pd.read_sql(session.query(Star).filter(Star.population_id == k).statement, session.bind) 
        mean = np.mean(df.distance)
        
    elif method == 'pandas_query_distances':
        df = pd.read_sql(session.query(Star.distance).filter(Star.population_id == k).statement, session.bind) 
        mean = np.mean(df.distance)
        
    elif method == 'sql_function':
        mean = session.query(func.avg(Star.distance)).filter(Star.population_id == k).one()[0]
        
    
    end['time'] = wallclock.time()
    end['memory'] = measure.heap().size

    time = (end['time']-start['time']) * 1e3
    memory = (end['memory']-start['memory']) / (1024.0*1024.0)

    print "Estimated mean distance = ", mean

    print "Wallclock time spent = ", np.round(time,1), "milliseconds"
    print "Memory used = ",np.round(memory,1), "Mb"
    
    del stars, distances, mean

    return time, memory

In [15]:
t, m = estimate_mean(method='query_stars')

Estimated mean distance =  2.99972073362
Wallclock time spent =  2982.2 milliseconds
Memory used =  222.1 Mb


It's faster to query for just the distance, not the whole star:

In [16]:
t, m = estimate_mean(method='query_distances')

Estimated mean distance =  2.99972073362
Wallclock time spent =  548.3 milliseconds
Memory used =  9.3 Mb


What if we read the stars, or distances, into a `pandas` data frame, and used that? 

In [17]:
t, m = estimate_mean(method='pandas_query_stars')

Estimated mean distance =  2.99972073362
Wallclock time spent =  571.7 milliseconds
Memory used =  0.1 Mb


In [18]:
t, m = estimate_mean(method='pandas_query_distances')

Estimated mean distance =  2.99972073362
Wallclock time spent =  487.1 milliseconds
Memory used =  0.1 Mb


It looks as though `pandas` is doing something clever with the memory, and speeding up the array extraction as well.

How does this compare with doing the math in SQL directly?

In [19]:
t, m = estimate_mean(method='sql_function')

Estimated mean distance =  2.99972073362
Wallclock time spent =  159.8 milliseconds
Memory used =  0.1 Mb


## Conclusions

The slowest thing you can do is loop over the rows in a table, pulling out each row as its own object. In our test case it was a factor of ~5 faster to just extract the distances. `pandas` allows you to avoid this difference: it was only 25% slower to read the whole table into a `pandas` data frame and compute a mean as it was to extract and work on just the column we wanted. Single column extraction and averaging was about 50% faster with `pandas`.

Calculating an average was about 3 times faster using the built-in SQL `avg` function than doing a `np.mean()` on an extracted column (whether or not it is read into a `pandas` dataframe). This procedure also used 2000 times less memory than reading all the objects into memory, and 100 times less than just reading in the column we needed. Using `pandas` seemed to be as memory-efficient as doing the calculation in the database, though!

In [20]:
try: os.remove(dbfile)
except: pass